# Drug plates

In [ ]:
library(ggplot2)
library(extrafont)
library(stringr)
library(heatmaply)
library(ggrepel)
library(reticulate)
library(gridExtra)
library(robustbase)

In [ ]:
# Load external Python functions
source_python("reticulate_functions.py")
# Load extra fonts
loadfonts()
# Set this to true for faster compilation or false for more precise results and all outputs
TEST_MODE = T

In [ ]:
# Number of permutations used for empirical p-value computations
PERM_NB_ITER = ifelse(TEST_MODE, 20, 2000)

In [ ]:
# Customize ggplot appearance

# Change theme
customTheme <- theme_light() + 
               theme(panel.grid.minor=element_blank(), text=element_text(size=17, family="Arial", colour = "#333333"),
                     line=element_line(colour = "#333333"), legend.background = element_rect(fill=alpha('#CCCCCC', 0.1)), legend.key = element_blank())

# Change default colors
scale_colour_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_colour_gradient(...), 
        viridis = scale_colour_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))
}
scale_color_continuous <- scale_colour_continuous

scale_fill_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_fill_gradient(...), 
        viridis = scale_fill_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))

}

cemm_pal = colorRampPalette(c("#5A463C", "#008CAD", "#40B9D4", "#D4ECF2", "#D2323C", "#F8B100", "#DFDC00"))
cust_pal = colorRampPalette(c("#008CAD", "#40B9D4", "#D4ECF2", "#F8B100", "#C00000", "#2D0000"))
yolla_pal = colorRampPalette(c('#FD0F91', '#C00000', '#2D0000'))
scale_fill_discrete <- function (..., type = "Cust", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "fill") 
{
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else if (type == "Yolla"){
        discrete_scale(aesthetics, "Yolla", yolla_pal, na.value = na.value, ...)
    } else if (type == "Cust"){
        discrete_scale(aesthetics, "Cust", cust_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}

scale_color_discrete <- function (..., type = "Cust", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "colour") {
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else if (type == "Cust"){
        discrete_scale(aesthetics, "Cust", cust_pal, na.value = na.value, ...)
    } else if (type == "Yolla"){
        discrete_scale(aesthetics, "Yolla", yolla_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", scales::hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}
scale_colour_discrete <- scale_color_discrete

noGridTheme <- function(...){
    theme(panel.grid.major=element_blank(), axis.text.x=element_text(size=12), axis.text.y=element_text(size=12),
                      axis.line=element_line(color="#333333", size = 0.2), panel.border = element_blank(), ...)
}

darkTheme <- function(...){
    theme(panel.background = element_rect(fill = '#333333'), plot.background = element_rect(fill = '#333333'), 
          axis.line=element_line(color="#CCCCCC", size = 0.2), 
          text=element_text(size=17, family="Arial", colour = "#CCCCCC"),
          line=element_line(colour = "#CCCCCC"))
}

theme_set(customTheme)

options(repr.plot.width=10, repr.plot.height=10)

## NK92

In [ ]:
# Fix t-SNE layout and random number generation
set.seed(38)

In [ ]:
NK = read.csv("./allImages_NK_Drugs.csv", header=T)

In [ ]:
Layout = read.csv("./DrugPlateLayout.csv", header=T)
Layout$Well <- paste0(Layout$Row, str_pad(Layout$Column, 2, pad = "0"))

In [ ]:
NK$Drug = sapply(NK$Metadata_Well, function(x) Layout$Drug[x == Layout$Well])
levels(NK$Drug)[levels(NK$Drug)=="Y27"] <- "Y-27632"
NK$Concentration = sapply(NK$Metadata_Well, function(x) Layout$Concentration[x == Layout$Well])

In [ ]:
gpNK = ggplot(NK[!is.na(NK$Count_FilteredNucleus),]) + geom_histogram(aes(Count_FilteredNucleus, fill = as.factor(Metadata_Row)), binwidth=2) +
       scale_fill_discrete(name="Row")
gpNK

if(!TEST_MODE){
    ggsave(filename = "Fig/NK_count.pdf", plot = gpNK)
}

In [ ]:
gpNK = ggplot(NK[!is.na(NK$Mean_FilterCytoplasm_AreaShape_Area),]) + geom_histogram(aes(Mean_FilterCytoplasm_AreaShape_Area, 
                                                                                    fill = as.factor(Metadata_Row)), binwidth=100) +
       scale_fill_discrete(name="Row")
gpNK
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_area.pdf", plot = gpNK)
}

### Filtering

In [ ]:
FILT_MAX_INT_DNA = 0.05 # Remove empty images and small DNA precipitations
FILT_MIN_CELLS = 4 # 8 seems safe from distribution and images, 3 seems in poor shape
FILT_NB_MAX_NA_IMAGE = 10
FILT_MAX_CORR = 0.6 # Keep uncorrelated variables
dimUMAP = 3

In [ ]:
ftToKeep = 1:dim(NK)[2]
# Make sure that the fields are numeric
ftToKeep <- ftToKeep[which(sapply(NK[,ftToKeep], class) == "numeric")]
# Remove execution time and count features
ftToKeep <- ftToKeep[grep("(Execution)|(Count)", colnames(NK)[ftToKeep], invert = T)]

In [ ]:
# Store all remaining features before filtering for downstream comparison to selected set
preFiltFt = colnames(NK)[ftToKeep]

In [ ]:
# Remove wells with low max DNA intensity
fieldToKeep <- which(NK$ImageQuality_MaxIntensity_DNA >= FILT_MAX_INT_DNA)
# Remove wells with low cell count
fieldToKeep <- fieldToKeep[NK[fieldToKeep,]$Count_FilteredNucleus >= FILT_MIN_CELLS]

In [ ]:
# Few bad quality pictures are generating a lot of missing values and are removed
fieldToKeep <- fieldToKeep[rowSums(is.na(NK[fieldToKeep,ftToKeep])) < FILT_NB_MAX_NA_IMAGE]
# Remove remaining features with missing values
ftToKeep <- ftToKeep[colSums(is.na(NK[fieldToKeep,ftToKeep])) == 0] 
# Remove constant columns
indWT = NK[fieldToKeep,]$Drug == "WT"
ftToKeep <- ftToKeep[sapply(NK[fieldToKeep,ftToKeep], function(x) mad(x) != 0)]
ftToKeep <- ftToKeep[sapply(NK[fieldToKeep[indWT],ftToKeep], function(x) mad(x) != 0)]

In [ ]:
length(apply(NK[fieldToKeep,ftToKeep], 2, function(x) mad(x) != 0))

In [ ]:
length(sapply(NK[fieldToKeep,ftToKeep], function(x) mad(x) != 0))

In [ ]:
length(sapply(NK[fieldToKeep,ftToKeep], function(x) mad(x) != 0))
length(ftToKeep)

In [ ]:
print(dim(NK))
print(length(ftToKeep))
print(length(fieldToKeep))

In [ ]:
# Approximate normal distribution
transfLog <- function (x){
    minVal = min(x)
    return(log(x+1-minVal))
}

# Center and scale on control values
transfNorm <- function(x, y){
    return((x - median(y))/mad(y))
}

In [ ]:
transformedNK = apply(NK[fieldToKeep, ftToKeep], 2, transfLog)
transformedNK = apply(transformedNK, 2, function(x) transfNorm(x, x[indWT]))

In [ ]:
# Order features from biggest mad to smallest mad
# Since features have mad(WT) = 1, it means that we rank features by how more variable they are
# for drug perturbations than for WT
orderFt = rev(order(apply(transformedNK, 2, mad)))

In [ ]:
uncorrFt = uncorrelate(transformedNK, orderCol = orderFt-1, threshold = FILT_MAX_CORR)
uncorrFt = unlist(uncorrFt) + 1

In [ ]:
transformedNK = transformedNK[,uncorrFt]

### Look at which types of features are kept

In [ ]:
catChannel = c("CorrDNA", "CorrActin", "CorrPerf")
table(rowSums(sapply(catChannel, function(x) grepl(x, colnames(transformedNK)))))
dtCat = data.frame(CountIni = sort(colSums(sapply(catChannel, function(x) grepl(x, preFiltFt)))), 
                   Count = sort(colSums(sapply(catChannel, function(x) grepl(x, colnames(transformedNK))))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) + ylim(c(0,675)) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_SelecFt_Channel.pdf", plot = gp, width = 10)
}

In [ ]:
catObjects = c("ActinGranules", "FilterCytoplasm", "ShrunkenCytoplasm", "FilteredNucleus", "PerfGranules")
table(rowSums(sapply(catObjects, function(x) grepl(x, colnames(transformedNK)))))
dtCat = data.frame(CountIni = colSums(sapply(catObjects, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catObjects, function(x) grepl(x, colnames(transformedNK)))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) + ylim(c(0,510)) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_SelecFt_Object.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Threshold", "Granularity", "ImageQuality", "Texture", "Distance", "AreaShape", "RadialDistribution", "Neighbors", 
            "Correlation", "Intensity", "Overlap", "Location")
which(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedNK)))) == 0) # All features are covered

dtCat = data.frame(CountIni = colSums(sapply(catType, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catType, function(x) grepl(x, colnames(transformedNK)))))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_SelecFt_Type_All.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Correlation", "Intensity")
table(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedNK)))))

dtCount = as.data.frame(sapply(catType, function(x) grepl(x, colnames(transformedNK))))
dtCount$Other = !apply(dtCount, 1, any)
dtCountIni = as.data.frame(sapply(catType, function(x) grepl(x, preFiltFt)))
dtCountIni$Other = !apply(dtCountIni, 1, any)
dtCat = data.frame(CountIni = colSums(dtCountIni), 
                   Count = colSums(dtCount))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_SelecFt_Type_Short.pdf", plot = gp, width = 10)
}

In [ ]:
# Export list of features kept
if(!TEST_MODE){
    write.table(colnames(transformedNK), file = "Tab/NK_list_features.csv", col.names = F, row.names = F)
}

### Look at the morphological distribution of the fields of view

#### UMAP visualizations

In [ ]:
umTNK = umap(transformedNK, min_dist = 0.1, neighbors = 10, n = dimUMAP, metric = "euclidean")
umTNK = as.data.frame(umTNK)
names(umTNK) = c("UMAP1", "UMAP2")
umTNK$Row <- as.factor(NK$Metadata_Row[fieldToKeep])
umTNK$Col <- as.factor(NK$Metadata_Column[fieldToKeep])
umTNK$Drug <- as.factor(NK$Drug[fieldToKeep])
umTNK$Conc <- as.factor(NK$Concentration[fieldToKeep])
umTNK$URL <- as.factor(NK$URL_Actin[fieldToKeep])

In [ ]:
# Order levels to separate DMSO and WT from drugs
lvDMSO = which(levels(umTNK$Drug) == "DMSO")
lvWT = which(levels(umTNK$Drug) == "WT")
umTNK$Drug <- factor(umTNK$Drug, levels = c("DMSO", rev(levels(umTNK$Drug)[-c(lvDMSO, lvWT)]), "WT"))

In [ ]:
umTNK$Shape = ifelse(umTNK$Drug %in% c("DMSO", "WT"), "2", "1")

In [ ]:
gp <- ggplot(umTNK) + geom_point(aes(UMAP1, UMAP2, color = Drug, shape = Shape)) + 
                      guides(color = guide_legend(reverse = TRUE, 
                               override.aes = list(shape = ifelse(levels(umTNK$Drug) %in% c("DMSO", "WT"), 1, 16))),
                             shape = F) + scale_shape_manual(values = c(16,1))
gp

#### Alternate drug colors

    umTNK$Drug = factor(umTNK$Drug, levels = c('DMSO', 'Latrunculin B', 'Cytochalasin D', 'CK869', 'Y-27632', 
                                               'Wiskostatin', 'SMIFH2', 'Jasplaknolide', 'Blebbistatin', 'WT'))
    levels(umTNK$Drug)

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_UMAP_Drugs.pdf", plot = gp, width = 10)
}

In [ ]:
umXlim = ggplot_build(gp)$layout$panel_scales_x[[1]]$range$range
umYlim = ggplot_build(gp)$layout$panel_scales_y[[1]]$range$range

In [ ]:
umTNK$DrugOrControl <- as.factor(ifelse(umTNK$Drug == "DMSO", "DMSO", ifelse(umTNK$Drug == "WT", "WT", "Drug")))

gp <- ggplot(umTNK) + geom_point(aes(UMAP1, UMAP2, color = DrugOrControl, shape = Shape)) + 
    theme(legend.title = element_blank()) + guides(color = guide_legend(reverse = TRUE, 
                               override.aes = list(shape = ifelse(levels(umTNK$DrugOrControl) %in% c("DMSO", "WT"), 1, 16))),
                             shape = F) + scale_shape_manual(values = c(16,1))
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_UMAP_Control.pdf", plot = gp, width = 10)
}

In [ ]:
# Visualize drug perturbations one by one
getDrugUmap <- function(drugPert){
    drug = levels(umTNK$Drug)[drugPert]
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$Drug == drug],]$Metadata_Row[1]

    umDrug = umTNK[umTNK$Row == rowPert,]
    cols = cust_pal(length(levels(umTNK$Drug)))[c(length(levels(umTNK$Drug)), drugPert, 1)]
    names(cols) = c("WT", drug, "DMSO")
    gp <- ggplot(umDrug) + geom_point(aes(UMAP1, UMAP2, color = Drug, shape = Shape)) + 
        scale_color_manual(values = cols) + ggtitle(drug) +
        theme(legend.title = element_blank(), legend.position = "bottom", legend.key.size = unit(0.08,"cm"),
              legend.text = element_text(size = 10), legend.spacing.x = unit(0.08,"cm"), text=element_text(size=12)) +
        xlim(umXlim) + ylim(umYlim) + scale_shape_manual(values = c(16,1)) +
        guides(color = guide_legend(override.aes = list(shape = c(1, 16, 1))), shape = F)
    return(gp)
}

In [ ]:
if(!TEST_MODE){
    gpl = lapply(2:(length(levels(umTNK$Drug))-1), getDrugUmap)
    gp = do.call("grid.arrange", c(gpl, ncol=4))
    gp
    ggsave(filename = "Fig/NK_UMAP_by_drug.pdf", plot = gp, width = 10)
}

#### UMAP of morphological features

In [ ]:
umNKFTfit = umap_fit(t(transformedNK), min_dist = 0.05, neighbors = 8, n = 2, metric = "euclidean")
umNKFT = umap_transform(umNKFTfit, t(transformedNK))
umNKFT = as.data.frame(umNKFT)
names(umNKFT) = c("UMAP1", "UMAP2")

In [ ]:
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Correlation", "Intensity")
umNKFT$type = apply(as.data.frame(sapply(catType, function(x) 
    grepl(x, colnames(transformedNK)))), 1, function(x) paste(catType[which(x)],collapse='-'))
umNKFT$type[umNKFT$type == ""] <- NA

In [ ]:
catChannel = c("CorrDNA", "CorrActin", "CorrPerf")
umNKFT$channel = apply(as.data.frame(sapply(catChannel, function(x) 
    grepl(x, colnames(transformedNK)))), 1, function(x) paste(catChannel[which(x)],collapse='-'))
umNKFT$channel[umNKFT$channel == ""] <- NA

In [ ]:
gp <- ggplot(umNKFT) + geom_point(aes(UMAP1, UMAP2, col = type, shape = channel), size = 3)
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_UMAP_Features.pdf", gp, width = 10)
}

In [ ]:
confoundingDF = rbind(Count_Filtered_Nuclei = NK$Count_FilteredNucleus, 
                      Count_All_Nuclei = NK$Count_Nucleus,
                      Row = NK$Metadata_Row,
                      Column = NK$Metadata_Column,
                      Well = NK$Metadata_Well)[,fieldToKeep]
confoundingUMAP = umap_transform(umNKFTfit, confoundingDF)
confoundingUMAP = as.data.frame(confoundingUMAP)
names(confoundingUMAP) = c("UMAP1", "UMAP2")
confoundingUMAP$type = rownames(confoundingDF)
confoundingUMAP$size = 4

In [ ]:
confoundingAndKept = rbind(confoundingUMAP, cbind(umNKFT[,1:2], type = "Kept feature", size = 2))

In [ ]:
gp <- ggplot(confoundingAndKept) + geom_point(aes(UMAP1, UMAP2, col = type, size = size)) +
      guides(size=F) + scale_size_identity()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_UMAP_Features_Confounding.pdf", gp, width = 10)
}

#### Distances to WT cells

In [ ]:
# Compute the median Robust Mahalanobis Distance (RMD) between drugs
drugRMD <- function(drugPert){
    # Find row containing this drug and split wells between WT and drug
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    setUmapDrug = umTNK[NK[fieldToKeep,]$Drug == drugPert,1:dimUMAP]
    setUmapWT = umTNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert],1:dimUMAP]
    
    # Ensure that we have enough points to compute distance
    if ((dim(setUmapDrug)[1] < 2*dimUMAP)|(dim(setUmapWT)[1] < 2*dimUMAP)){
            print(drugPert)
            return(NA)
    }
    
    # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
    mcdWT = covMcd(setUmapWT)
    RMD = median(apply(setUmapDrug, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov)))
    return(RMD)
}

In [ ]:
shuffDrugRMD <- function(drugPert, nbRep = PERM_NB_ITER){
    # Find row containing this drug and shuffle wells between WT and drug
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    setUmapDrug = umTNK[NK[fieldToKeep,]$Drug == drugPert,1:dimUMAP]
    setUmapWT = umTNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert],1:dimUMAP]
    setUMAP = rbind(setUmapWT,setUmapDrug)
    shuffleRMD <- function(notUsed){
        shuffSetUMAP = rbind(setUmapWT,setUmapDrug)[sample(nrow(setUMAP)),]
        # Take random subsets of corresponding sizes
        shuffSetDrug = shuffSetUMAP[1:nrow(setUmapDrug),]
        shuffSetWT = shuffSetUMAP[(nrow(setUmapDrug)+1):(nrow(setUmapDrug)+nrow(setUmapWT)),]
        
        # Ensure that we have enough points to compute distance
        if ((dim(shuffSetDrug)[1] < 2*dimUMAP)|(dim(shuffSetWT)[1] < 2*dimUMAP)){
                print(drugPert)
                return(NA)
        }
        
        # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
        mcdWT = covMcd(shuffSetWT)
        RMD = median(apply(shuffSetDrug, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov)))
        return(RMD)
    }
    return(sapply(1:nbRep, shuffleRMD))
}

In [ ]:
# Similar procedure for DMSO against WT
dmsoRMD <- function(rowDMSO){
    setUmapDMSO = umTNK[(NK[fieldToKeep,]$Drug == "DMSO")&(NK[fieldToKeep,]$Metadata_Row == rowDMSO),1:dimUMAP]
    setUmapWT = umTNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowDMSO],1:dimUMAP]
   
    # Ensure that we have enough points to compute distance
    if ((dim(setUmapDMSO)[1] < 2*dimUMAP)|(dim(setUmapWT)[1] < 2*dimUMAP)){
            return(NA)
    }
    
    # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
    mcdWT = covMcd(setUmapWT)
    RMD = median(apply(setUmapDMSO, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov)))
    return(RMD)
}
shuffDmsoRMD <- function(rowDMSO, nbRep = PERM_NB_ITER){
    # Find row containing this drug and shuffle wells between WT and drug
    setUmapDMSO = umTNK[(NK[fieldToKeep,]$Drug == "DMSO")&(NK[fieldToKeep,]$Metadata_Row == rowDMSO),1:dimUMAP]
    setUmapWT = umTNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowDMSO],1:dimUMAP]
    setUMAP = rbind(setUmapWT,setUmapDMSO)
    shuffleRMD <- function(notUsed){
        shuffSetUMAP = rbind(setUmapWT,setUmapDMSO)[sample(nrow(setUMAP)),]
        # Take random subsets of corresponding sizes
        shuffSetDMSO = shuffSetUMAP[1:nrow(setUmapDMSO),]
        shuffSetWT = shuffSetUMAP[(nrow(setUmapDMSO)+1):(nrow(setUmapDMSO)+nrow(setUmapWT)),]
        
        # Ensure that we have enough points to compute distance
        if ((dim(shuffSetDMSO)[1] < 2*dimUMAP)|(dim(shuffSetWT)[1] < 2*dimUMAP)){
                return(NA)
        }
        
        # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
        mcdWT = covMcd(shuffSetWT)
        RMD = median(apply(shuffSetDMSO, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov)))
        return(RMD)
    }
    return(sapply(1:nbRep, shuffleRMD))
}

In [ ]:
drugRMD_NK = sapply(levels(NK$Drug), drugRMD)

In [ ]:
shuffDrugRMD_NK = sapply(levels(NK$Drug), shuffDrugRMD)

In [ ]:
getRMPV <- function(x){
    ecdfRMD = ecdf(x[-1])
    return(1 - ecdfRMD(x[1]))
}
dfRMPV = data.frame(RMPV = apply(rbind(drugRMD_NK, shuffDrugRMD_NK), 2, getRMPV))

In [ ]:
dfRMPV$Drug = rownames(dfRMPV)
dfRMPV$Strength = drugRMD_NK
dfRMPV = dfRMPV[!dfRMPV$Drug %in% c("DMSO", "WT"),]
dfRMPV$adjRMPV = p.adjust(dfRMPV$RMPV, method = "fdr")

In [ ]:
gp <- ggplot(dfRMPV) + geom_point(aes(adjRMPV, Strength, color = Drug)) + geom_vline(xintercept = 0.05, color="#CCCCCC", linetype="dashed")
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_RMPV_Drugs.pdf", gp, width = 10)
}

In [ ]:
dmsoRMD_NK = sapply(1:8, dmsoRMD)
shuffDmsoRMD_NK = sapply(1:8, shuffDmsoRMD)

In [ ]:
dmsoRMPV = data.frame(RMPV = apply(rbind(dmsoRMD_NK, shuffDmsoRMD_NK), 2, getRMPV))
dmsoRMPV$Row = rownames(dmsoRMPV)
dmsoRMPV$Strength = dmsoRMD_NK
dmsoRMPV$adjRMPV = p.adjust(dmsoRMPV$RMPV, method = "fdr")

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_RMPV_DMSO.pdf", plot = gp, width = 10)
}

### Concentration-dependency

In [ ]:
# Compute the Robust Mahalanobis Distance (RMD) for all images of a given drug at a given concentration
drugConcRMD <- function(drugPert, concPert){
    # Find row containing this drug and split wells between WT and drug
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    pertInd = (umTNK$Drug == drugPert)&(umTNK$Conc == concPert)
    setUmapDrug = umTNK[pertInd,1:dimUMAP]
    setUmapWT = umTNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert],1:dimUMAP]
    
    # Ensure that we have enough points to compute distance
    if ((dim(setUmapDrug)[1] < 2*dimUMAP)|(dim(setUmapWT)[1] < 2*dimUMAP)){
            print(c(drugPert, concPert))
            return(NA)
    }
    
    # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
    mcdWT = covMcd(setUmapWT)
    RMD = apply(setUmapDrug, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov))
    return(RMD)
}

In [ ]:
# Which drugs should be studied?
sigDrugs = rownames(dfRMPV)[dfRMPV$adjRMPV < 0.05]
# Classify concentration per drug as follows
concentrationLevels = c("Low", "Mid", "High")

getAllDrugConcRMD <- function(d, rename = T){
    dConc = levels(droplevels(umTNK[umTNK$Drug == d,]$Conc))
    dList = sapply(dConc, function(c) drugConcRMD(d,c), simplify = F)
    if (rename){
        names(dList) <- concentrationLevels[1:length(dConc)]
    }
    return(dList)
}
sigDrugConcRMD = sapply(sigDrugs, getAllDrugConcRMD, simplify = F)

In [ ]:
getSublist <- function(x, y){
    i = which(names(x) == y)
    # If y is not a sublist of x, i is the empty integer (not null, not false but with zero-length)
    if (length(i)){
        return(x[[i]])
}}

concRMPV = sapply(concentrationLevels, function(y) unlist(sapply(sigDrugConcRMD, function(x) getSublist(x, y))))
concRMPV = as.data.frame(Reduce(rbind, Map(cbind, RMPV = concRMPV, Concentration = names(concRMPV))))
concRMPV$RMPV = as.numeric(concRMPV$RMPV)
concRMPV$Concentration = factor(concRMPV$Concentration, levels = concentrationLevels)

In [ ]:
gp <- ggplot(concRMPV) + geom_violin(aes(Concentration, RMPV, fill = Concentration)) + 
                         coord_flip() + theme(legend.title = element_blank()) + guides(fill = guide_legend(reverse = TRUE)) +
                         scale_fill_discrete(type = "Yolla")
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_ConcEffect_All.pdf", plot = gp, width = 10)
}

In [ ]:
drugsConcRMPV = sapply(sigDrugs, function(x) getAllDrugConcRMD(x, rename = F), simplify = F)

for (i in 1:length(drugsConcRMPV)){
    # Retrieve drug name and RMPV values
    drugName = names(drugsConcRMPV)[i]
    drugConcRMPV = drugsConcRMPV[[i]]
    
    # Structure into wide data frame
    concRMPV = as.data.frame(Reduce(rbind, Map(cbind, RMPV = drugConcRMPV, Concentration = names(drugConcRMPV))))
    concRMPV$RMPV = as.numeric(concRMPV$RMPV)
    # Reorder concentrations in decreasing numerical order
    concRMPV$Concentration = factor(concRMPV$Concentration, 
                                levels = as.character(sort(as.numeric(levels(concRMPV$Concentration)))))

    # Represent distribution per concentration for all significantly changed drug
    gp <- ggplot(concRMPV) + geom_violin(aes(Concentration, RMPV, fill = Concentration)) + 
                             coord_flip() + theme(legend.title = element_blank()) + guides(fill = guide_legend(reverse = T)) +
                             scale_fill_discrete(type = "Yolla")
    if(!TEST_MODE){
        ggsave(filename = paste("Fig/NK", drugName, "ConcEffect.pdf", sep = "_"), plot = gp, width = 10)
    }
}

### Morphological changes upon perturbation

In [ ]:
# Associate categories and colors to features
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Intensity")
colType = cust_pal(length(catType)+1)

dtCount = as.data.frame(sapply(catType, function(x) grepl(x, colnames(transformedNK))))
dtCount$Other = !apply(dtCount, 1, any)
 
ftCat = as.factor(apply(dtCount, 1, function(x) colnames(dtCount)[which(x)]))

# Order levels so that "Other" is last
lvOther = which(levels(ftCat) == "Other")
ftCat = factor(ftCat, levels = c(levels(ftCat)[-lvOther], "Other"))
       
# Corresponding colors
ftCol = colType[ftCat]

In [ ]:
# Plot number of images / categories (useful to get fill color legend)
gp <- ggplot(data.frame(Category = ftCat), aes(fill = Category)) + geom_bar(aes(Category))
if(!TEST_MODE){
    ggsave(filename = "Fig/NK_CatCount.pdf", plot = gp, width = 10)
}

In [ ]:
changedFtDrug <- function(drugPert){
    # Plate row on which the drug is
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    
    # Morphological features for the drug and the WT control on the same row
    setDrug = transformedNK[NK[fieldToKeep,]$Drug == drugPert,]
    setWT = transformedNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert],]
    
    medianDrug = apply(setDrug, 2, median)
    medianWT = apply(setWT, 2, median)
    medianChange = medianDrug - medianWT
    
    # Compute difference to WT
    ftToDisplay = order(abs(medianChange), decreasing = T)[1:12]
    dtChange = data.frame(Difference = medianChange[ftToDisplay])
    dtChange$Feature = rownames(dtChange)
    dtChange$Feature = factor(dtChange$Feature, levels = dtChange$Feature[order(dtChange$Difference)]) 
    dtChange$Category = ftCol[ftToDisplay]

    gp <- ggplot(dtChange) + geom_bar(aes(Feature, weight = Difference, fill = Category), color = "#CCCCCC") + 
                   coord_flip() + ylab("Difference to WT") + scale_fill_identity() +
                   theme(legend.position="none", plot.margin=unit(c(0,5,0,0), "mm"))
    
    if(!TEST_MODE){
        ggsave(filename = paste("Fig/NK", drugPert, "ChangeFt.pdf", sep = "_"), plot = gp, width = 10)
    }
}

In [ ]:
sapply(sigDrugs, changedFtDrug)

## Is concentration-dependent effect acting on similar or distinct features?
Example of Y27

In [ ]:
changedFtY27 <- function(concPert){  
    drugPert = "Y27"
    
    # Plate row on which the drug is
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    
    # Morphological features for the drug and the WT control on the same row
    setDrug = transformedNK[(NK[fieldToKeep,]$Drug == drugPert)&(NK[fieldToKeep,]$Concentration == concPert),]
    setWT = transformedNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert],]
    
    medianDrug = apply(setDrug, 2, median)
    medianWT = apply(setWT, 2, median)
    medianChange = medianDrug - medianWT
    
    # Compute difference to WT
    ftToDisplay = order(abs(medianChange), decreasing = T)[1:12]
    dtChange = data.frame(Difference = medianChange[ftToDisplay])
    dtChange$Feature = rownames(dtChange)
    dtChange$Feature = factor(dtChange$Feature, levels = dtChange$Feature[order(dtChange$Difference)]) 
    dtChange$Category = ftCol[ftToDisplay]

    gp <- ggplot(dtChange) + geom_bar(aes(Feature, weight = Difference, fill = Category), color = "#CCCCCC") + 
                   coord_flip() + ylab("Difference to WT") + scale_fill_identity() +
                   theme(legend.position="none", plot.margin=unit(c(0,5,0,0), "mm"))
    
    if(!TEST_MODE){
        ggsave(filename = paste("Fig/NK_Y27", concPert, "ChangeFt.pdf", sep = "_"), plot = gp, width = 10)
    }
}

In [ ]:
sapply(c(5,10,25), changedFtY27)

## Jurkat

In [ ]:
# Fix t-SNE layout and random number generation
set.seed(38)

In [ ]:
JK = read.csv("./allImages_Jurkat_Drugs.csv", header=T)

In [ ]:
JK$Drug = sapply(JK$Metadata_Well, function(x) Layout$Drug[x == Layout$Well])
levels(JK$Drug)[levels(JK$Drug)=="Y27"] <- "Y-27632"
JK$Concentration = sapply(JK$Metadata_Well, function(x) Layout$Concentration[x == Layout$Well])

In [ ]:
gpJK = ggplot(JK[!is.na(JK$Count_FilteredNucleus),]) + geom_histogram(aes(Count_FilteredNucleus, fill = as.factor(Metadata_Row)), binwidth=2) +
       scale_fill_discrete(name="Row")
gpJK
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_count.pdf", plot = gpJK)
}

In [ ]:
gpJK = ggplot(JK[!is.na(JK$Mean_FilterCytoplasm_AreaShape_Area),]) + geom_histogram(aes(Mean_FilterCytoplasm_AreaShape_Area, 
                                                                                    fill = as.factor(Metadata_Row)), binwidth=100) +
       scale_fill_discrete(name="Row")
gpJK
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_area.pdf", plot = gpJK)
}

In [ ]:
FILT_MAX_INT_DNA = 0.05 # Remove empty images and small DNA precipitations
FILT_MIN_CELLS = 2 # 1 cell 
# >= 352 NAs 5/6 bad images
# >= 82 NAs 4/6 bad images
# < 82 NAs 3/6 bad images + 1 single-cell image
FILT_NB_MAX_NA_IMAGE = 83


In [ ]:
colnames(JK)[ftToKeep][grep("(Execution)|(Count)|(Location)", colnames(JK)[ftToKeep])]

In [ ]:
ftToKeep = 1:dim(JK)[2]
# Make sure that the fields are numeric
ftToKeep <- ftToKeep[which(sapply(JK[,ftToKeep], class) == "numeric")]
# Remove execution time and count features
ftToKeep <- ftToKeep[grep("(Execution)|(Count)", colnames(JK)[ftToKeep], invert = T)]

In [ ]:
# Store all remaining features before filtering for downstream comparison to selected set
preFiltFt = colnames(JK)[ftToKeep]

In [ ]:
# See Time-resolved mapping of genetic interactions to model rewiring of signaling pathways. Heigwer et al. (2018).
# Remove wells with low max DNA intensity
fieldToKeep <- which(JK$ImageQuality_MaxIntensity_DNA >= FILT_MAX_INT_DNA)
# Remove wells with low cell count
fieldToKeep <- fieldToKeep[!is.na(JK[fieldToKeep,]$Count_FilteredNucleus)]
fieldToKeep <- fieldToKeep[JK[fieldToKeep,]$Count_FilteredNucleus >= FILT_MIN_CELLS]

In [ ]:
# Few bad quality pictures are generating a lot of missing values and are removed
fieldToKeep <- fieldToKeep[rowSums(is.na(JK[fieldToKeep,ftToKeep])) < FILT_NB_MAX_NA_IMAGE]
# Remove remaining features with missing values
ftToKeep <- ftToKeep[colSums(is.na(JK[fieldToKeep,ftToKeep])) == 0] 
# Remove constant columns
indWT = JK[fieldToKeep,]$Drug == "WT"
ftToKeep <- ftToKeep[sapply(JK[fieldToKeep,ftToKeep], function(x) mad(x) != 0)]
ftToKeep <- ftToKeep[sapply(JK[fieldToKeep[indWT],ftToKeep], function(x) mad(x) != 0)]

In [ ]:
print(dim(JK))
print(length(ftToKeep))
print(length(fieldToKeep))

In [ ]:
transformedJK = apply(JK[fieldToKeep, ftToKeep], 2, transfLog)
transformedJK = apply(transformedJK, 2, function(x) transfNorm(x, x[indWT]))

In [ ]:
# Order features from biggest mad to smallest mad
# Since features have mad(WT) = 1, it means that we rank features by how more variable they are
# for drug perturbations than for WT
orderFt = rev(order(apply(transformedJK, 2, mad)))

In [ ]:
uncorrFt = uncorrelate(transformedJK, orderCol = orderFt-1, threshold = FILT_MAX_CORR)
uncorrFt = unlist(uncorrFt) + 1

In [ ]:
transformedJK = transformedJK[,uncorrFt]

### Look at which types of features are kept

In [ ]:
catChannel = c("CorrDNA", "CorrActin", "CorrPerf")
table(rowSums(sapply(catChannel, function(x) grepl(x, colnames(transformedJK)))))
dtCat = data.frame(CountIni = sort(colSums(sapply(catChannel, function(x) grepl(x, preFiltFt)))), 
                   Count = sort(colSums(sapply(catChannel, function(x) grepl(x, colnames(transformedJK))))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) + ylim(c(0,675)) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_SelecFt_Channel.pdf", plot = gp, width = 10)
}

In [ ]:
catObjects = c("ActinGranules", "FilterCytoplasm", "ShrunkenCytoplasm", "FilteredNucleus", "PerfGranules")
table(rowSums(sapply(catObjects, function(x) grepl(x, colnames(transformedJK)))))
dtCat = data.frame(CountIni = colSums(sapply(catObjects, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catObjects, function(x) grepl(x, colnames(transformedJK)))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) + ylim(c(0,510)) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_SelecFt_Object.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Threshold", "Granularity", "ImageQuality", "Texture", "Distance", "AreaShape", "RadialDistribution", "Neighbors", 
            "Correlation", "Intensity", "Overlap", "Location")
which(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedJK)))) == 0) # All features are covered

dtCat = data.frame(CountIni = colSums(sapply(catType, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catType, function(x) grepl(x, colnames(transformedJK)))))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_SelecFt_Type_All.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Correlation", "Intensity")
table(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedJK)))))

dtCount = as.data.frame(sapply(catType, function(x) grepl(x, colnames(transformedJK))))
dtCount$Other = !apply(dtCount, 1, any)
dtCountIni = as.data.frame(sapply(catType, function(x) grepl(x, preFiltFt)))
dtCountIni$Other = !apply(dtCountIni, 1, any)
dtCat = data.frame(CountIni = colSums(dtCountIni), 
                   Count = colSums(dtCount))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_SelecFt_Type_Short.pdf", plot = gp, width = 10)
}

In [ ]:
# Export list of features kept
if(!TEST_MODE){
    write.table(colnames(transformedJK), file = "Tab/JK_list_features.csv", col.names = F, row.names = F)
}

### Look at the morphological distribution of the fields of view

#### UMAP visualizations

In [ ]:
umTJK = umap(transformedJK, min_dist = 0.1, neighbors = 10, n = dimUMAP, metric = "euclidean")
umTJK = as.data.frame(umTJK)
names(umTJK) = c("UMAP1", "UMAP2")
umTJK$Row <- as.factor(JK$Metadata_Row[fieldToKeep])
umTJK$Col <- as.factor(JK$Metadata_Column[fieldToKeep])
umTJK$Drug <- as.factor(JK$Drug[fieldToKeep])
umTJK$Conc <- as.factor(JK$Concentration[fieldToKeep])
umTJK$URL <- as.factor(JK$URL_Actin[fieldToKeep])

In [ ]:
# Order levels to separate DMSO and WT from drugs
lvDMSO = which(levels(umTJK$Drug) == "DMSO")
lvWT = which(levels(umTJK$Drug) == "WT")
umTJK$Drug <- factor(umTJK$Drug, levels = c("DMSO", rev(levels(umTJK$Drug)[-c(lvDMSO, lvWT)]), "WT"))

In [ ]:
umTJK$Shape = ifelse(umTJK$Drug %in% c("DMSO", "WT"), "2", "1")

In [ ]:
gp <- ggplot(umTJK) + geom_point(aes(UMAP1, UMAP2, color = Drug, shape = Shape)) + 
                      guides(color = guide_legend(reverse = TRUE, 
                               override.aes = list(shape = ifelse(levels(umTJK$Drug) %in% c("DMSO", "WT"), 1, 16))),
                             shape = F) + scale_shape_manual(values = c(16,1))
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_UMAP_Drugs.pdf", plot = gp, width = 10)
}

In [ ]:
umXlim = ggplot_build(gp)$layout$panel_scales_x[[1]]$range$range
umYlim = ggplot_build(gp)$layout$panel_scales_y[[1]]$range$range

In [ ]:
umTJK$DrugOrControl <- as.factor(ifelse(umTJK$Drug == "DMSO", "DMSO", ifelse(umTJK$Drug == "WT", "WT", "Drug")))

gp <- ggplot(umTJK) + geom_point(aes(UMAP1, UMAP2, color = DrugOrControl, shape = Shape)) + 
    theme(legend.title = element_blank()) + guides(color = guide_legend(reverse = TRUE, 
                               override.aes = list(shape = ifelse(levels(umTJK$DrugOrControl) %in% c("DMSO", "WT"), 1, 16))),
                             shape = F) + scale_shape_manual(values = c(16,1))
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_UMAP_Control.pdf", plot = gp, width = 10)
}

In [ ]:
# Visualize drug perturbations one by one
getDrugUmap <- function(drugPert){
    drug = levels(umTJK$Drug)[drugPert]
    rowPert = JK[fieldToKeep[JK[fieldToKeep,]$Drug == drug],]$Metadata_Row[1]

    umDrug = umTJK[umTJK$Row == rowPert,]
    cols = cust_pal(length(levels(umTJK$Drug)))[c(length(levels(umTJK$Drug)), drugPert, 1)]
    names(cols) = c("WT", drug, "DMSO")
    gp <- ggplot(umDrug) + geom_point(aes(UMAP1, UMAP2, color = Drug, shape = Shape)) + 
        scale_color_manual(values = cols) + ggtitle(drug) +
        theme(legend.title = element_blank(), legend.position = "bottom", legend.key.size = unit(0.08,"cm"),
              legend.text = element_text(size = 10), legend.spacing.x = unit(0.08,"cm"), text=element_text(size=12)) +
        xlim(umXlim) + ylim(umYlim) + scale_shape_manual(values = c(16,1)) +
        guides(color = guide_legend(override.aes = list(shape = c(1, 16, 1))), shape = F)
    return(gp)
}

In [ ]:
if(!TEST_MODE){
    gpl = lapply(2:(length(levels(umTJK$Drug))-1), getDrugUmap)
    gp = do.call("grid.arrange", c(gpl, ncol=4))
    gp
    ggsave(filename = "Fig/JK_UMAP_by_drug.pdf", plot = gp, width = 10)
}

#### UMAP of morphological features

In [ ]:
umJKFTfit = umap_fit(t(transformedJK), min_dist = 0.05, neighbors = 8, n = 2, metric = "euclidean")
umJKFT = umap_transform(umJKFTfit, t(transformedJK))
umJKFT = as.data.frame(umJKFT)
names(umJKFT) = c("UMAP1", "UMAP2")

In [ ]:
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Correlation", "Intensity")
umJKFT$type = apply(as.data.frame(sapply(catType, function(x) 
    grepl(x, colnames(transformedJK)))), 1, function(x) paste(catType[which(x)],collapse='-'))
umJKFT$type[umJKFT$type == ""] <- NA

In [ ]:
catChannel = c("CorrDNA", "CorrActin", "CorrPerf")
umJKFT$channel = apply(as.data.frame(sapply(catChannel, function(x) 
    grepl(x, colnames(transformedJK)))), 1, function(x) paste(catChannel[which(x)],collapse='-'))
umJKFT$channel[umJKFT$channel == ""] <- NA

In [ ]:
gp <- ggplot(umJKFT) + geom_point(aes(UMAP1, UMAP2, col = type, shape = channel), size = 3)
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_UMAP_Features.pdf", gp, width = 10)
}

In [ ]:
confoundingDF = rbind(Count_Filtered_Nuclei = JK$Count_FilteredNucleus, 
                      Count_All_Nuclei = JK$Count_Nucleus,
                      Row = JK$Metadata_Row,
                      Column = JK$Metadata_Column,
                      Well = JK$Metadata_Well)[,fieldToKeep]
confoundingUMAP = umap_transform(umJKFTfit, confoundingDF)
confoundingUMAP = as.data.frame(confoundingUMAP)
names(confoundingUMAP) = c("UMAP1", "UMAP2")
confoundingUMAP$type = rownames(confoundingDF)
confoundingUMAP$size = 4

In [ ]:
confoundingAndKept = rbind(confoundingUMAP, cbind(umJKFT[,1:2], type = "Kept feature", size = 2))

In [ ]:
gp <- ggplot(confoundingAndKept) + geom_point(aes(UMAP1, UMAP2, col = type, size = size)) +
      guides(size=F) + scale_size_identity()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_UMAP_Features_Confounding.pdf", gp, width = 10)
}

#### Distances to WT cells

In [ ]:
# Compute the median Robust Mahalanobis Distance (RMD) between drugs
# See https://e-archivo.uc3m.es/bitstream/handle/10016/24613/ws201710.pdf
drugRMD <- function(drugPert){
    # Find row containing this drug and split wells between WT and drug
    rowPert = JK[fieldToKeep[JK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    setUmapDrug = umTJK[JK[fieldToKeep,]$Drug == drugPert,1:dimUMAP]
    setUmapWT = umTJK[which(indWT)[JK[fieldToKeep[indWT],]$Metadata_Row == rowPert],1:dimUMAP]
    
    # Ensure that we have enough points to compute distance
    if ((dim(setUmapDrug)[1] < 2*dimUMAP)|(dim(setUmapWT)[1] < 2*dimUMAP)){
            print(drugPert)
            return(NA)
    }
    
    # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
    mcdWT = covMcd(setUmapWT)
    RMD = median(apply(setUmapDrug, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov)))
    return(RMD)
}

In [ ]:
# See The Multidimensional Perturbation Value. Hutz et al. (2013).
shuffDrugRMD <- function(drugPert, nbRep = PERM_NB_ITER){
    # Find row containing this drug and shuffle wells between WT and drug
    rowPert = JK[fieldToKeep[JK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    setUmapDrug = umTJK[JK[fieldToKeep,]$Drug == drugPert,1:dimUMAP]
    setUmapWT = umTJK[which(indWT)[JK[fieldToKeep[indWT],]$Metadata_Row == rowPert],1:dimUMAP]
    setUMAP = rbind(setUmapWT,setUmapDrug)
    shuffleRMD <- function(notUsed){
        shuffSetUMAP = rbind(setUmapWT,setUmapDrug)[sample(nrow(setUMAP)),]
        # Take random subsets of corresponding sizes
        shuffSetDrug = shuffSetUMAP[1:nrow(setUmapDrug),]
        shuffSetWT = shuffSetUMAP[(nrow(setUmapDrug)+1):(nrow(setUmapDrug)+nrow(setUmapWT)),]
        
        # Ensure that we have enough points to compute distance
        if ((dim(shuffSetDrug)[1] < 2*dimUMAP)|(dim(shuffSetWT)[1] < 2*dimUMAP)){
                print(drugPert)
                return(NA)
        }
        
        # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
        mcdWT = covMcd(shuffSetWT)
        RMD = median(apply(shuffSetDrug, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov)))
        return(RMD)
    }
    return(sapply(1:nbRep, shuffleRMD))
}

In [ ]:
# Similar procedure for DMSO against WT
dmsoRMD <- function(rowDMSO){
    setUmapDMSO = umTJK[(JK[fieldToKeep,]$Drug == "DMSO")&(JK[fieldToKeep,]$Metadata_Row == rowDMSO),1:dimUMAP]
    setUmapWT = umTJK[which(indWT)[JK[fieldToKeep[indWT],]$Metadata_Row == rowDMSO],1:dimUMAP]
    
    # Ensure that we have enough points to compute distance
    if ((dim(setUmapDMSO)[1] < 2*dimUMAP)|(dim(setUmapWT)[1] < 2*dimUMAP)){
            return(NA)
    }
    
    # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
    mcdWT = covMcd(setUmapWT)
    RMD = median(apply(setUmapDMSO, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov)))
    return(RMD)
}
shuffDmsoRMD <- function(rowDMSO, nbRep = PERM_NB_ITER){
    # Find row containing this drug and shuffle wells between WT and drug
    setUmapDMSO = umTJK[(JK[fieldToKeep,]$Drug == "DMSO")&(JK[fieldToKeep,]$Metadata_Row == rowDMSO),1:dimUMAP]
    setUmapWT = umTJK[which(indWT)[JK[fieldToKeep[indWT],]$Metadata_Row == rowDMSO],1:dimUMAP]
    setUMAP = rbind(setUmapWT,setUmapDMSO)
    shuffleRMD <- function(notUsed){
        shuffSetUMAP = rbind(setUmapWT,setUmapDMSO)[sample(nrow(setUMAP)),]
        # Take random subsets of corresponding sizes
        shuffSetDMSO = shuffSetUMAP[1:nrow(setUmapDMSO),]
        shuffSetWT = shuffSetUMAP[(nrow(setUmapDMSO)+1):(nrow(setUmapDMSO)+nrow(setUmapWT)),]
        
        # Ensure that we have enough points to compute distance
        if ((dim(shuffSetDMSO)[1] < 2*dimUMAP)|(dim(shuffSetWT)[1] < 2*dimUMAP)){
                return(NA)
        }
        
        # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
        mcdWT = covMcd(shuffSetWT)
        RMD = median(apply(shuffSetDMSO, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov)))
        return(RMD)
    }
    return(sapply(1:nbRep, shuffleRMD))
}

In [ ]:
drugRMD_JK = sapply(levels(JK$Drug), drugRMD)

In [ ]:
shuffDrugRMD_JK = sapply(levels(JK$Drug), shuffDrugRMD)

In [ ]:
getRMPV <- function(x){
    ecdfRMD = ecdf(x[-1])
    return(1 - ecdfRMD(x[1]))
}
dfRMPV = data.frame(RMPV = apply(rbind(drugRMD_JK, shuffDrugRMD_JK), 2, getRMPV))

In [ ]:
dfRMPV$Drug = rownames(dfRMPV)
dfRMPV$Strength = drugRMD_JK
dfRMPV = dfRMPV[!dfRMPV$Drug %in% c("DMSO", "WT"),]
dfRMPV$adjRMPV = p.adjust(dfRMPV$RMPV, method = "fdr")

In [ ]:
gp <- ggplot(dfRMPV) + geom_point(aes(adjRMPV, Strength, color = Drug)) + geom_vline(xintercept = 0.05, color="#CCCCCC", linetype="dashed")
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_RMPV_Drugs.pdf", plot = gp, width = 10)
}

In [ ]:
dmsoRMD_JK = sapply(1:8, dmsoRMD)
shuffDmsoRMD_JK = sapply(1:8, shuffDmsoRMD)

In [ ]:
dmsoRMPV = data.frame(RMPV = apply(rbind(dmsoRMD_JK, shuffDmsoRMD_JK), 2, getRMPV))
dmsoRMPV$Row = rownames(dmsoRMPV)
dmsoRMPV$Strength = dmsoRMD_JK
dmsoRMPV$adjRMPV = p.adjust(dmsoRMPV$RMPV, method = "fdr")

In [ ]:
gp <- ggplot(dmsoRMPV) + geom_point(aes(adjRMPV, Strength, color = Row)) + geom_vline(xintercept = 0.05, color="#CCCCCC", linetype="dashed")
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_RMPV_DMSO.pdf", plot = gp, width = 10)
}

### Concentration-dependency

In [ ]:
# Compute the Robust Mahalanobis Distance (RMD) for all images of a given drug at a given concentration
drugConcRMD <- function(drugPert, concPert){
    # Find row containing this drug and split wells between WT and drug
    rowPert = JK[fieldToKeep[JK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    pertInd = (umTJK$Drug == drugPert)&(umTJK$Conc == concPert)
    setUmapDrug = umTJK[pertInd,1:dimUMAP]
    setUmapWT = umTJK[which(indWT)[JK[fieldToKeep[indWT],]$Metadata_Row == rowPert],1:dimUMAP]
    
    # Ensure that we have enough points to compute distance
    if ((dim(setUmapDrug)[1] < 2*dimUMAP)|(dim(setUmapWT)[1] < 2*dimUMAP)){
            print(c(drugPert, concPert))
            return(NA)
    }
    
    # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
    mcdWT = covMcd(setUmapWT)
    RMD = apply(setUmapDrug, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov))
    return(RMD)
}

In [ ]:
# Which drugs should be studied?
sigDrugs = rownames(dfRMPV)[dfRMPV$adjRMPV < 0.05]
# Classify concentration per drug as follows
concentrationLevels = c("Low", "Mid", "High")

getAllDrugConcRMD <- function(d, rename = T){
    dConc = levels(droplevels(umTJK[umTJK$Drug == d,]$Conc))
    dList = sapply(dConc, function(c) drugConcRMD(d,c), simplify = F)
    if (rename){
        names(dList) <- concentrationLevels[1:length(dConc)]
    }
    return(dList)
}
sigDrugConcRMD = sapply(sigDrugs, getAllDrugConcRMD, simplify = F)

In [ ]:
getSublist <- function(x, y){
    i = which(names(x) == y)
    # If y is not a sublist of x, i is the empty integer (not null, not false but with zero-length)
    if (length(i)){
        return(x[[i]])
}}

concRMPV = sapply(concentrationLevels, function(y) unlist(sapply(sigDrugConcRMD, function(x) getSublist(x, y))))
concRMPV = as.data.frame(Reduce(rbind, Map(cbind, RMPV = concRMPV, Concentration = names(concRMPV))))
concRMPV$RMPV = as.numeric(concRMPV$RMPV)
concRMPV$Concentration = factor(concRMPV$Concentration, levels = concentrationLevels)

In [ ]:
gp <- ggplot(concRMPV) + geom_violin(aes(Concentration, RMPV, fill = Concentration)) + 
                         coord_flip() + theme(legend.title = element_blank()) + guides(fill = guide_legend(reverse = TRUE)) +
                         scale_fill_discrete(type = "Yolla")
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_ConcEffect_All.pdf", plot = gp, width = 10)
}

In [ ]:
drugsConcRMPV = sapply(sigDrugs, function(x) getAllDrugConcRMD(x, rename = F), simplify = F)

for (i in 1:length(drugsConcRMPV)){
    # Retrieve drug name and RMPV values
    drugName = names(drugsConcRMPV)[i]
    drugConcRMPV = drugsConcRMPV[[i]]
    
    # Structure into wide data frame
    concRMPV = as.data.frame(Reduce(rbind, Map(cbind, RMPV = drugConcRMPV, Concentration = names(drugConcRMPV))))
    concRMPV$RMPV = as.numeric(concRMPV$RMPV)
    # Reorder concentrations in decreasing numerical order
    concRMPV$Concentration = factor(concRMPV$Concentration, 
                                levels = as.character(sort(as.numeric(levels(concRMPV$Concentration)))))
    
    # Represent distribution per concentration for all significantly changed drug
    gp <- ggplot(concRMPV) + geom_violin(aes(Concentration, RMPV, fill = Concentration)) + 
                             coord_flip() + theme(legend.title = element_blank()) + guides(fill = guide_legend(reverse = TRUE)) +
                             scale_fill_discrete(type = "Yolla")
    if(!TEST_MODE){
        ggsave(filename = paste("Fig/JK", drugName, "ConcEffect.pdf", sep = "_"), plot = gp, width = 10)
    }
}

### Morphological changes upon perturbation

In [ ]:
# Associate categories and colors to features
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Intensity")
colType = cust_pal(length(catType)+1)

dtCount = as.data.frame(sapply(catType, function(x) grepl(x, colnames(transformedJK))))
dtCount$Other = !apply(dtCount, 1, any)
 
ftCat = as.factor(apply(dtCount, 1, function(x) colnames(dtCount)[which(x)]))

# Order levels so that "Other" is last
lvOther = which(levels(ftCat) == "Other")
ftCat = factor(ftCat, levels = c(levels(ftCat)[-lvOther], "Other"))
       
# Corresponding colors
ftCol = colType[ftCat]

In [ ]:
# Plot number of images / categories (useful to get fill color legend)
gp <- ggplot(data.frame(Category = ftCat), aes(fill = Category)) + geom_bar(aes(Category))
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_CatCount.pdf", plot = gp, width = 10)
}

In [ ]:
changedFtDrug <- function(drugPert){  
    # Plate row on which the drug is
    rowPert = JK[fieldToKeep[JK[fieldToKeep,]$Drug == drugPert],]$Metadata_Row[1]
    
    # Morphological features for the drug and the WT control on the same row
    setDrug = transformedJK[JK[fieldToKeep,]$Drug == drugPert,]
    setWT = transformedJK[which(indWT)[JK[fieldToKeep[indWT],]$Metadata_Row == rowPert],]
    
    medianDrug = apply(setDrug, 2, median)
    medianWT = apply(setWT, 2, median)
    medianChange = medianDrug - medianWT
    
    # Compute difference to WT
    ftToDisplay = order(abs(medianChange), decreasing = T)[1:12]
    dtChange = data.frame(Difference = medianChange[ftToDisplay])
    dtChange$Feature = rownames(dtChange)
    dtChange$Feature = factor(dtChange$Feature, levels = dtChange$Feature[order(dtChange$Difference)]) 
    dtChange$Category = ftCol[ftToDisplay]

    gp <- ggplot(dtChange) + geom_bar(aes(Feature, weight = Difference, fill = Category), color = "#CCCCCC") + 
                   coord_flip() + ylab("Difference to WT") + scale_fill_identity() +
                   theme(legend.position="none", plot.margin=unit(c(0,5,0,0), "mm"))
    
    if(!TEST_MODE){
        ggsave(filename = paste("Fig/JK", drugPert, "ChangeFt.pdf", sep = "_"), plot = gp, width = 10)
    }
}

In [ ]:
sapply(sigDrugs, changedFtDrug)

In [ ]:
sessionInfo()